In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets

import requests
from urllib.parse import quote
import tkinter as tk
from tkinter import ttk

import config # Imports my private API Key

# Team Casimir Funk Nutritional Content Notebook
In this notebook, we create the our nutritional content functions.

In [585]:
def handle_query(query, food_df_dict, api_key, num_results = 10):
    encoded_query = quote(query)
    page_number = 1
    page_size = num_results

    url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={api_key}&query={encoded_query}&pageSize={page_size}&pageNumber={page_number}'

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")
    
    food_df = pd.DataFrame(data['foods'])
    
    food_df_dict['food_df'] = food_df
    display(food_df.head())

In [587]:
def get_nutritional_content(food_df, idx = 0, computation = False):
    desired_cols = ['fdcId', 'description', 'brandOwner', 'brandName', 'marketCountry']
    avail_cols = [col for col in desired_cols if col in food_df.columns]
    
    detail_df = pd.DataFrame(food_df.loc[idx, avail_cols])
    detail_df.rename(columns = {idx : 'Details'}, inplace = True)
    
    
    nutritional_df = pd.DataFrame(food_df.loc[idx, 'foodNutrients'])
    
    desired_cols2 = ['nutrientName', 'value', 'unitName', 'percentDailyValue']
    avail_cols2 = [col for col in desired_cols2 if col in nutritional_df.columns]
    nutritional_df = nutritional_df[avail_cols2]
    
    if computation:
        return nutritional_df
    else:
        display(detail_df)
        display(nutritional_df)

In [600]:
def interactive_query(api_key):
    food_df_dict = {}
    widget = interactive(handle_query, 
                               food_df_dict = widgets.fixed(food_df_dict),
                               query = widgets.Text(description='Enter Query Here'),  
                               num_results = (1, 20, 1),
                               api_key = widgets.fixed(api_key)
                        )
    display(widget)
    return food_df_dict

In [607]:
def interactive_get_nc(food_df_dict):
    widget = interactive(get_nutritional_content, food_df = widgets.fixed(food_df_dict['food_df']), 
                  idx = widgets.IntSlider(value = 0, min = 0, max = len(food_df_dict['food_df']) - 1, step = 1, description='Food Index'), 
                         computation = widgets.fixed(False)
                        )
    display(widget)